In [ ]:
# HTMLHeaderTextSplitter
# MarkdownHeaderTextSplitter와 개념적으로 유사한 HTMLHeaderTextSplitter는 텍스트를 요소 수준에서 분할하고 
# 각 헤더에 대한 메타데이터를 추가하는 "구조 인식" 청크 생성기입니다.
# 이는 각 청크와 "관련된" 메타데이터를 추가합니다.
# HTMLHeaderTextSplitter는 요소별로 청크를 반환하거나 동일한 메타데이터를 가진 요소를 결합할 수 있으며
#
# (a) 관련 텍스트를 의미론적으로 (대략적으로) 그룹화하고
# (b) 문서 구조에 인코딩된 컨텍스트 풍부한 정보를 보존하는 것을 목표로 합니다.

# %pip install -qU langchain-text-splitters

In [2]:
from langchain_text_splitters import HTMLHeaderTextSplitter

html_string = """
<!DOCTYPE html>
<html>
<body>
    <div>
        <h1>Foo</h1>
        <p>Some intro text about Foo.</p>
        <div>
            <h2>Bar main section</h2>
            <p>Some intro text about Bar.</p>
            <h3>Bar subsection 1</h3>
            <p>Some text about the first subtopic of Bar.</p>
            <h3>Bar subsection 2</h3>
            <p>Some text about the second subtopic of Bar.</p>
        </div>
        <div>
            <h2>Baz</h2>
            <p>Some text about Baz</p>
        </div>
        <br>
        <p>Some concluding text about Foo</p>
    </div>
</body>
</html>
"""

headers_to_split_on = [
    ("h1", "Header 1"),  # 분할할 헤더 태그와 해당 헤더의 이름을 지정합니다.
    ("h2", "Header 2"),
    ("h3", "Header 3"),
]

# 지정된 헤더를 기준으로 HTML 텍스트를 분할하는 HTMLHeaderTextSplitter 객체를 생성합니다.
html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# HTML 문자열을 분할하여 결과를 html_header_splits 변수에 저장합니다.
html_header_splits = html_splitter.split_text(html_string)

# 출력
for header in html_header_splits:
    print(f"[page_content]:\n{header.page_content}\n")
    print(f"[metadata]:\n{header.metadata}", end="\n=============================\n\n")
    

[page_content]:
Foo

[metadata]:
{}

[page_content]:
Some intro text about Foo.  
Bar main section Bar subsection 1 Bar subsection 2

[metadata]:
{'Header 1': 'Foo'}

[page_content]:
Some intro text about Bar.

[metadata]:
{'Header 1': 'Foo', 'Header 2': 'Bar main section'}

[page_content]:
Some text about the first subtopic of Bar.

[metadata]:
{'Header 1': 'Foo', 'Header 2': 'Bar main section', 'Header 3': 'Bar subsection 1'}

[page_content]:
Some text about the second subtopic of Bar.

[metadata]:
{'Header 1': 'Foo', 'Header 2': 'Bar main section', 'Header 3': 'Bar subsection 2'}

[page_content]:
Baz

[metadata]:
{'Header 1': 'Foo'}

[page_content]:
Some text about Baz

[metadata]:
{'Header 1': 'Foo', 'Header 2': 'Baz'}

[page_content]:
Some concluding text about Foo

[metadata]:
{'Header 1': 'Foo'}



In [4]:
# 다른 splitter와 파이프라인으로 연결하고, 웹 URL에서 HTML을 로드하는 경우입니다.
# 웹 URL로부터 HTML 콘텐츠를 로드한 후, 이를 다른 splitter와 파이프라인으로 연결하여 처리하는 과정입니다.

from langchain_text_splitters import RecursiveCharacterTextSplitter

url = "https://news.naver.com"  # 분할할 텍스트의 URL을 지정합니다.

headers_to_split_on = [  # 분할할 HTML 헤더 태그와 해당 헤더의 이름을 지정합니다.
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("h4", "Header 4"),
]

# HTML 헤더를 기준으로 텍스트를 분할하는 HTMLHeaderTextSplitter 객체를 생성합니다.
html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# URL에서 텍스트를 가져와 HTML 헤더를 기준으로 분할합니다.
html_header_splits = html_splitter.split_text_from_url(url)

chunk_size = 500  # 텍스트를 분할할 청크의 크기를 지정합니다.
chunk_overlap = 30  # 분할된 청크 간의 중복되는 문자 수를 지정합니다.
text_splitter = RecursiveCharacterTextSplitter(  # 텍스트를 재귀적으로 분할하는 RecursiveCharacterTextSplitter 객체를 생성합니다.
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# HTML 헤더로 분할된 텍스트를 다시 청크 크기에 맞게 분할합니다.
splits = text_splitter.split_documents(html_header_splits)

# 분할된 텍스트 중 80번째부터 85번째까지의 청크를 출력합니다.
for header in splits[80:85]:
    print(f"{header.page_content}")
    print(f"{header.metadata}", end="\n=====================\n")

국정전환 'NO', 특검 'NO'…尹대통령 '그대로'  
박찬대 "尹, 몹시 실망스러운 회견, 변화 기회 스스로 걷어차"  
조국 "마이동풍, 동문서답, 오불관언"  
이전  
다음  
새로보기  
언론사별 심층기획  
구독  
기획보도
{'Header 3': '콘텐츠'}
대구MBC 05월 10일 11:02
{'Header 3': '콘텐츠', 'Header 4': '기획보도 대구MBC 05월 10일 11:02'}
[약손+] 앎으로 암을 극복, '위암 수술 후 건강관리' ⑦나이 그리고 소화제  
[날씨체크] 변덕이 죽 끓듯···맑다가 주말 또 '비 소식'  
[백투더투데이] "환자를 첫째로 생각하겠습니다" 1980년대 의과대학  
[토크와이드] 연체율 최대… ‘위기의 소상공인’  
이전  
다음  
구독  
뉴스 더 보기
{'Header 3': '콘텐츠'}
아이뉴스24 05월 10일 07:48
{'Header 3': '콘텐츠', 'Header 4': '뉴스 더 보기 아이뉴스24 05월 10일 07:48'}
그때는 '합치자' 읍소, 지금은 '떠나라' 윽박...네이버 뒤통수 친 日 [IT돋보기]  
[데스크칼럼] 겹겹으로 덮인 산속의 우주항공청  
[기자수첩] '헛바퀴' 주택공급 대책 언제까지  
"이제 반년 남았다"…100만 가입자 향해 달리는 '캐즐'  
이전  
다음  
구독  
윤 취임 2주년 회견
{'Header 3': '콘텐츠'}


In [5]:
# **한계**
# HTMLHeaderTextSplitter는 HTML 문서 간의 구조적 차이를 처리하려고 시도하지만, 때로는 특정 헤더를 누락할 수 있습니다.
# 예를 들어, 이 알고리즘은 헤더가 항상 관련 텍스트보다 "위"에 있는 노드, 즉 이전 형제 노드, 조상 노드 및 이들의 조합에 위치한다고 가정합니다.
#
# 다음 뉴스 기사(이 문서 작성 시점 기준)에서는 최상위 헤드라인의 텍스트가 "h1"으로 태그되어 있지만, 우리가 예상하는 텍스트 요소와는 별개의 하위 트리 에 있는 것을 볼 수 있습니다.
# 따라서 "h1" 요소와 관련 텍스트는 청크 메타데이터에 나타나지 않지만, 해당되는 경우 "h2"와 관련 텍스트는 볼 수 있습니다.

# 분할할 HTML 페이지의 URL을 지정합니다.
url = "https://www.cnn.com/2023/09/25/weather/el-nino-winter-us-climate/index.html"

headers_to_split_on = [
    ("h1", "Header 1"),  # 분할할 헤더 태그와 해당 헤더의 이름을 지정합니다.
    ("h2", "Header 2"),  # 분할할 헤더 태그와 해당 헤더의 이름을 지정합니다.
]

# 지정된 헤더를 기준으로 HTML 텍스트를 분할하는 HTMLHeaderTextSplitter 객체를 생성합니다.
html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# 지정된 URL의 HTML 페이지를 분할하여 결과를 html_header_splits 변수에 저장합니다.
html_header_splits = html_splitter.split_text_from_url(url)

# 분할된 결과를 출력합니다.
for header in html_header_splits:
    print(f"{header.page_content[:100]}")
    print(f"{header.metadata}", end="\n=====================\n")

CNN values your feedback  
1. How relevant is this ad to you?  
2. Did you encounter any technical i
{}
No two El Niño winters are the same, but many have temperature and precipitation trends in common.  
{'Header 2': 'What could this winter look like?'}
Ad Feedback  
Ad Feedback  
Ad Feedback  
Ad Feedback  
Ad Feedback  
Ad Feedback  
Live TV Listen W
{}
